In [39]:
import numpy as np
from numba import njit

In [46]:
@njit
def lu_compact(A):
  
    assert A.shape[0] == A.shape[1]

    U = A.astype(np.float64)
    n = A.shape[0]
    L = np.eye(n).astype(np.float64)

    for k in range(n - 1):
        for j in range(k + 1, n):
            L[j, k] = U[j, k] / U[k, k]
            U[j, k:n] -= L[j, k] * U[k, k:n]

    return L, U

In [43]:
@njit
def lu_inplace(A, L, U):
    
    assert A.shape[0] == A.shape[1]
    assert L.shape[0] == L.shape[1] == A.shape[0]
    assert U.shape[0] == U.shape[1] == A.shape[0]

    size = len(A)

    for i in range(size):
        for k in range(size):
            total = np.sum(L[i, 0:i] * U[0:i, k])
            U[i, k] = A[i, k] - total

        for k in range(size):
            if i == k:
                L[i, i] = 1.0
            else:
                total = np.sum(L[k, 0:i] * U[0:i, i])
                L[k, i] = (A[k, i] - total) / U[i, i]


def lu(A):
    
    L = np.zeros_like(A)
    U = np.zeros_like(A)

    lu_inplace(A, L, U)
    return L, U

In [44]:
A = np.array([[4., 12., 8., 4.],
         [1., 7., 18., 9.],
         [2., 9., 20., 20.],
         [3., 11., 15., 14.]])

L = np.array([[1, 0, 0, 0],
     [0.25, 1.0, 0.0, 0.0],
     [0.5, 0.75, 1.0, 0.0],
     [0.75, 0.5, 0.25, 1.0],
     ])

U = np.array([[4.0, 12.0, 8.0, 4.0],
     [0.0, 4.0, 16.0, 8.0],
     [0.0, 0.0, 4.0, 12.0],
     [0.0, 0.0, 0.0, 4.0]])

In [49]:
%timeit lu_compact(A)

7.28 µs ± 1.43 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [50]:
L = np.zeros_like(A)
U = np.zeros_like(A)
%timeit lu_inplace(A, L, U)

28.8 µs ± 9.09 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
%timeit lu(A)

43.2 µs ± 2.3 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
